In [22]:
%matplotlib inline
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
from os import listdir
from os.path import isfile, join
from __future__ import print_function
from numpy import median, diff

# SongFile:
### Fields
- beat_frames:  
- beat_times: 
- bpm: 
- bpm_string: 
- beat_length: 
- sample_times: 
- indices: 
- music_samples: 
- data: 
### Output
- data/{0}_beat_times.csv
- data/{0}_beat_frames.csv
- data/{0}_music_samples.csv

In [19]:
sample_rate = 22050*8
samples_per_second = 24 * 4
steps_per_bar = 12
class SongFile:
    def load_new(self):
        data, _ = librosa.load(self.music_file, sr=sample_rate)
        _, self.beat_frames = librosa.beat.beat_track(y=data, sr=sample_rate)
        self.beat_times = librosa.frames_to_time(self.beat_frames, sr=sample_rate)

        seconds = len(data) / sample_rate
        times = get_beats(self.beat_times, self.beat_frames)
        self.bpm = times[0][1]
        self.bpm_string = get_time_string(times)
        
        self.beat_length = 60. / self.bpm
        bps = self.bpm / 60
        # take 24 samples for each beat (need 3rds, 8ths)
        num_samples = int(bps * seconds * samples_per_second)
        beat_time = self.beat_length / samples_per_second
        
        sample_times = [self.beat_times[0] + (beat_time * i) for i in range(num_samples)]
        # only take samples where music still playing
        self.sample_times = [time for time in sample_times if round(time * sample_rate) < len(data)]
        self.indices = [round(time * sample_rate) for time in self.sample_times]
        self.music_samples = [data[index] for index in self.indices]
        self.data = data
    
    def __init__(self, pack, name, extension, force_new):
        self.pack = pack
        self.name = name
        self.extension = extension
        self.music_file = 'StepMania/Songs/{0}/{1}/{1}.{2}'.format(pack, name, extension)
        self.stepfile = 'StepMania/Songs/{0}/{1}/{1}.ssc'.format(pack, name)
        
        key = '{0}~{1}'.format(pack, name)
        if force_new or (not '{0}_beat_frames.csv'.format(key) in listdir('data')):
            print ('Calculating beats for {0}'.format(key))
            self.load_new()
                        
            print ('Saving calculated beats for {0}'.format(key))
            pd.DataFrame(self.beat_times).to_csv('data/{0}_beat_times.csv'.format(key), index=False)
            pd.DataFrame(self.beat_frames).to_csv('data/{0}_beat_frames.csv'.format(key), index=False)
            pd.DataFrame(self.music_samples).to_csv('data/{0}_music_samples.csv'.format(key), index=False)
        else:
            #print ('Loading beats from save for {0}'.format(key))
            self.beat_frames = pd.read_csv('data/{0}_beat_frames.csv'.format(key)).values.flatten()
            self.beat_times = pd.read_csv('data/{0}_beat_times.csv'.format(key)).values.flatten()
            self.music_samples = pd.read_csv('data/{0}_music_samples.csv'.format(key)).values.flatten()

# Some useful functions to load induvidual or lists of songs
- load_song(pack: String, pack: String, force_new: Bool)
- load_songs(songs: Array(Pair(String~pack, String~title)), force_new: Bool)
- load_all_songs(force_new: Bool)

In [7]:
def load_song(pack, title, force_new):
    folder_path = 'StepMania/Songs/{0}/{1}'.format(pack, title)
    song = next(file for file in listdir(folder_path) if file.endswith('.ogg') or file.endswith('.mp3'))
    extension = song.split('.')[1]
    key = '{0}~{1}'.format(pack, title)
    return SongFile(pack, title, extension, force_new)

def load_songs(songs, force_new):
    return {'{0}~{1}'.format(song[0], song[1]): load_song(song[0], song[1], force_new) for song in songs}

def load_all_songs(force_new):
    songs = [('In The Groove', song) for song in listdir('StepMania/Songs/In The Groove') if song != '.DS_Store']
    songs.extend([('a_test', song) for song in ['A', 'C', 'C']])
    return load_songs(songs, force_new)

# Functions to get bpm from song
- get_beats(beat_times: Array(Float), beat_frames: Array(Int))
- get_time_string(times: Array(Pair(Float~timeToStart, Float~bpm)))

In [8]:
def get_beats(beat_times, beat_frames):
    changes = []
    changes_time = []
    for i in range(len(beat_frames) - 1):
        changes.append(beat_frames[i + 1] - beat_frames[i])
        changes_time.append(beat_times[i + 1] - beat_times[i])

    sorted_changes = sorted(changes)
    median = sorted_changes[int(len(changes) / 2)]

    changes_counted = [abs(change - median) < 2 for change in changes]
    time_changes_counted = list(itertools.compress(changes_time, changes_counted))
    average = sum(time_changes_counted) / len(time_changes_counted)
    
    time_differences = []
    earliest_proper_beat = 1
    for i in range(1, len(beat_times) - 1):
        if changes_counted[i] & changes_counted[i - 1]:
            earliest_proper_beat = i
            break
            
    last_proper_beat = len(beat_times) -2
    for i in range(1, len(beat_times) - 1):
        if changes_counted[len(beat_times) - i - 1] & changes_counted[len(beat_times) - i - 2]:
            last_proper_beat = len(beat_times) - i - 1
            break
    
    time_differences = []
    buffer = 5
    for i in range(20):
        start_beat = earliest_proper_beat + buffer * i
        if changes_counted[start_beat] & changes_counted[start_beat - 1]:
            for j in range(20):
                end_beat = last_proper_beat - buffer * j
                if changes_counted[end_beat] & changes_counted[end_beat - 1]:
                    time_differences.append(beat_times[end_beat] - beat_times[start_beat])
        
    # get num beats, round, and make new average
    new_averages = [time_difference / round(time_difference / average) for time_difference in time_differences]
    #print (new_averages)
    new_averages.sort()
    num_averages = len(new_averages)
    #new_average = sum(new_averages[5:num_averages - 5]) / (num_averages - 10)
    new_average = new_averages[int(num_averages/2)]
    return [(0, 60./new_average)]
    
def get_time_string(times):
    time_strings = []
    for time in times:
        if time[0] != 0:
            time_strings.append(',')
        time_strings.append('{:}={:.5f}'.format(time[0], time[1]))
    return ''.join(time_strings)

# Helpers to test get_beats() above
- get_bpm(file: String)
- get_song_bpms()
- test_get_beats(song_data: Map(SongData))

In [9]:
def get_bpm(file):
    with open(file, "r") as ins:
        for line in ins:
            if line.startswith('#BPMS:'):
                result = re.search('#BPMS:(.*);', line)
                bpm_string = result.group(1)
                if len(bpm_string.split(',')) == 1:
                    return float(bpm_string.split('=')[1])
                return 0
    return 0

def get_song_bpms():
    songs = [song for song in listdir('StepMania/Songs/In The Groove')]
    songs.remove('.DS_Store')
    song_bpms = {}
    for song in songs:
        song_bpms['In The Groove~{0}'.format(song)] = get_bpm('StepMania/Songs/In The Groove/{0}/{0}.sm'.format(song))

    song_bpms['_test~A'] = 181.685
    song_bpms['_test~B'] = 140
    song_bpms['_test~C'] = 180
    return song_bpms

def test_get_beats(song_data):
    song_bpms = get_song_bpms()
    errors = []
    for key in song_data:
        song = song_data[key]
        real_beat = song_bpms[key]
        if real_beat != 0:
            prediced_beat = get_beats(song.beat_times, song.beat_frames)[0][1]
            for i in range (1,4):
                if abs((prediced_beat * (i + 1) / (i)) - real_beat) < abs(prediced_beat - real_beat):
                    prediced_beat *= (i + 1) / (i)
            errors.append(abs(prediced_beat - real_beat))
    print (sorted(['{0:.3f}'.format(error) for error in errors]))

#test_get_beats(song_data)

# Helpers to read .ssc and return notes and meta data
- get_notes_from_note_string(note_string)
- get_notes_and_metadata(file)
- get_song_steps()

In [10]:
regex_notes_with_metadata = '#NOTES:n     dance-single((?:(?!//-).)*);'
regex_metadata_split = ':n     (.*):n     (.*):n     (.*):n     (.*):n     (.*):(.*);'
def get_notes_from_note_string(note_string):
    measure_comments = re.findall('( )*//( )*measure( )*[0-9]*', note_string)
    note_strings_split = re.split(r'n', note_string)[1:-1]
    notes = []
    bar = []
    for row in note_strings_split:
        if len(row) == 4:
            bar.append(row)
        else:
            notes.append(bar)
            bar = []
    return note_strings_split

def get_notes_and_metadata(file):
    difficulty_map = {}
    with open(file) as txt:
        step_file = txt.read()
        step_file = step_file.replace('\n', 'n')
        notes_with_metadata_groups = re.finditer(regex_notes_with_metadata, step_file)
        for match in notes_with_metadata_groups:
            notes_with_metadata = match.group(0)
            split_data = re.search(regex_metadata_split, notes_with_metadata)
            difficulty = split_data.group(4)
            metadata = split_data.group(5)
            notes = get_notes_from_note_string(split_data.group(6))
            notes_with_metadata_map = {
                'DIFFICULTY': difficulty,
                'METADATA': metadata,
                'NOTES': notes,
            }
            difficulty_map[difficulty] = notes_with_metadata_map
    return difficulty_map

def get_song_steps():
    songs = [song for song in listdir('StepMania/Songs/In The Groove')]
    songs.remove('.DS_Store')
    song_steps = {}
    for song in songs:
        song_steps['In The Groove~{0}'.format(song)] = get_notes_and_metadata('StepMania/Songs/In The Groove/{0}/{0}.sm'.format(song))
    return song_steps

In [12]:
def write_song_header(output_stepfile, song):
    keys = ['VERSION', 'TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH']
    
    header_info = {
        'VERSION': 0.82,
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.beat_times[0],
        'SAMPLESTART': song.beat_times[0] + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("//---------------dance-single - ----------------", file=output_stepfile)
    keys = ['NOTEDATA', 'CHARTNAME', 'STEPSTYPE', 'DIFFICULTY', 'METER', 'RADARVALUES', 'BPMS']
        
    step_info = {
        'NOTEDATA': '',
        'CHARTNAME': 'Kommisar',
        'STEPSTYPE': 'dance-single',
        'DIFFICULTY': 'Expert',
        'METER': 9,
        'RADARVALUES': '0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0,0.234,0.292,0.008,0,0,211,212,1,0,0,0,0,0,0',
        'BPMS': song.bpm_string
    }
    for key in keys:
        print ("#{0}:{1};".format(key, str(step_info[key])), file=output_stepfile)
        
def write_notes_simple(output_stepfile, song):
    print ("#NOTES:", file=output_stepfile)
    
    for i in range(40):
        print ("0001\n0001\n0001\n0001\n,", file=output_stepfile)
    print ("0000;", file=output_stepfile)
    
def write_notes(output_stepfile, song):
    print ("#NOTES:", file=output_stepfile)
    
    samples = song.music_samples
    # take steps_per_bar / 4 samples per beat (steps_per_bar per bar)
    steps_per_beat = steps_per_bar / 4
    filter_ammount = samples_per_second / steps_per_beat
    
    absolute_samples = [samples[i] for i in range(len(samples)) if i % filter_ammount == 0]
    # show 1 / 3 of all notes
    cutoff_index = int(len(absolute_samples) / 3)
    cutoff = sorted(absolute_samples)[-cutoff_index]
    indices = [sample > cutoff for sample in absolute_samples]
    
    for i in range(len(indices)):
        if indices[i]:
            print ("0001", file=output_stepfile)
        else:
            print ("0000", file=output_stepfile)
        if i % steps_per_bar == 0 and i != 0:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    output_stepfile=open(song.stepfile, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    write_notes(output_stepfile, song)
    output_stepfile.close()

In [3]:
#song_data = load_all_songs(False)

In [1101]:
#song_data = load_all_songs(False)

import copy
song = load_song('a_test', 'A', True)
song2 = copy.deepcopy(song)
song3 = copy.deepcopy(song)
song2.name = 'A2'
song3.name = 'A3'
song2.stepfile = 'StepMania/Songs/{0}/{1}/{1}.ssc'.format('a_test', 'A2')
song3.stepfile = 'StepMania/Songs/{0}/{1}/{1}.ssc'.format('a_test', 'A3')

Calculating beats for a_test~A


/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Saving calculated beats for a_test~A


In [1102]:
step_song(song)
#step_song(song2)
#step_song(song3)

In [20]:
#song = load_song('a_test', 'A', True)
#song2 = load_song('a_test', 'Fire', True)
song3 = load_song('a_test', 'WeBelongTogether', True)
song4 = load_song('a_test', 'CallMeBaby', True)

Calculating beats for a_test~WeBelongTogether


/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Saving calculated beats for a_test~WeBelongTogether
Calculating beats for a_test~CallMeBaby
Saving calculated beats for a_test~CallMeBaby


In [23]:
backup1 = copy.deepcopy(song)
backup2 = copy.deepcopy(song2)
backup3 = copy.deepcopy(song3)
backup4 = copy.deepcopy(song4)

In [26]:
y_harmonic, y_percussive = librosa.effects.hpss(song.data)

In [46]:
# Beat track on the percussive signal
beat_frames_old = song.indices
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sample_rate)

In [44]:
# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=song.data, sr=sample_rate, hop_length=512, n_mfcc=13)

In [31]:
# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

In [45]:
# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.feature.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

ParameterError: Invalid index set: [72704.0, 73593.0, 74482.0, 75371.0, 76260.0, 77150.0, 78039.0, 78928.0, 79817.0, 80706.0, 81595.0, 82484.0, 83373.0, 84263.0, 85152.0, 86041.0, 86930.0, 87819.0, 88708.0, 89597.0, 90486.0, 91375.0, 92265.0, 93154.0, 94043.0, 94932.0, 95821.0, 96710.0, 97599.0, 98488.0, 99378.0, 100267.0, 101156.0, 102045.0, 102934.0, 103823.0, 104712.0, 105601.0, 106490.0, 107380.0, 108269.0, 109158.0, 110047.0, 110936.0, 111825.0, 112714.0, 113603.0, 114493.0, 115382.0, 116271.0, 117160.0, 118049.0, 118938.0, 119827.0, 120716.0, 121606.0, 122495.0, 123384.0, 124273.0, 125162.0, 126051.0, 126940.0, 127829.0, 128718.0, 129608.0, 130497.0, 131386.0, 132275.0, 133164.0, 134053.0, 134942.0, 135831.0, 136721.0, 137610.0, 138499.0, 139388.0, 140277.0, 141166.0, 142055.0, 142944.0, 143833.0, 144723.0, 145612.0, 146501.0, 147390.0, 148279.0, 149168.0, 150057.0, 150946.0, 151836.0, 152725.0, 153614.0, 154503.0, 155392.0, 156281.0, 157170.0, 158059.0, 158948.0, 159838.0, 16072

In [34]:
# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sample_rate)

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:482: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return array(a, dtype, copy=False, order=order)


In [35]:
# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.feature.sync(chromagram, beat_frames, aggregate=np.median)

In [36]:
# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [40]:
print (len(beat_features))
print (len(beat_features[0]))

38
384


In [43]:
len(beat_frames)

37760

In [51]:
librosa.samples_to_frames(beat_frames_old)

[72704.0,
 73593.0,
 74482.0,
 75371.0,
 76260.0,
 77150.0,
 78039.0,
 78928.0,
 79817.0,
 80706.0,
 81595.0,
 82484.0,
 83373.0,
 84263.0,
 85152.0,
 86041.0,
 86930.0,
 87819.0,
 88708.0,
 89597.0,
 90486.0,
 91375.0,
 92265.0,
 93154.0,
 94043.0,
 94932.0,
 95821.0,
 96710.0,
 97599.0,
 98488.0,
 99378.0,
 100267.0,
 101156.0,
 102045.0,
 102934.0,
 103823.0,
 104712.0,
 105601.0,
 106490.0,
 107380.0,
 108269.0,
 109158.0,
 110047.0,
 110936.0,
 111825.0,
 112714.0,
 113603.0,
 114493.0,
 115382.0,
 116271.0,
 117160.0,
 118049.0,
 118938.0,
 119827.0,
 120716.0,
 121606.0,
 122495.0,
 123384.0,
 124273.0,
 125162.0,
 126051.0,
 126940.0,
 127829.0,
 128718.0,
 129608.0,
 130497.0,
 131386.0,
 132275.0,
 133164.0,
 134053.0,
 134942.0,
 135831.0,
 136721.0,
 137610.0,
 138499.0,
 139388.0,
 140277.0,
 141166.0,
 142055.0,
 142944.0,
 143833.0,
 144723.0,
 145612.0,
 146501.0,
 147390.0,
 148279.0,
 149168.0,
 150057.0,
 150946.0,
 151836.0,
 152725.0,
 153614.0,
 154503.0,
 155392.

In [48]:
beat_frames

array([  142,   308,   475,   641,   807,   974,  1141,  1308,  1475,
        1642,  1809,  2058,  2223,  2388,  2554,  2711,  2897,  3046,
        3223,  3386,  3571,  3762,  3953,  4143,  4309,  4464,  4598,
        4718,  4852,  4998,  5144,  5308,  5477,  5643,  5780,  5977,
        6140,  6310,  6444,  6555,  6717,  6860,  6973,  7113,  7261,
        7390,  7524,  7644,  7797,  7977,  8130,  8301,  8445,  8590,
        8727,  8934,  9090,  9224,  9383,  9603,  9765,  9889, 10060,
       10230, 10399, 10601, 10768, 10936, 11114, 11271, 11448, 11644,
       11815, 11977, 12148, 12314, 12481, 12649, 12815, 12980, 13145,
       13312, 13478, 13645, 13811, 13978, 14144, 14311, 14476, 14645,
       14811, 14977, 15143, 15311, 15477, 15645, 15790, 15978, 16145,
       16311, 16477, 16645, 16811, 16978, 17143, 17312, 17479, 17645,
       17811, 17979, 18145, 18312, 18477, 18646, 18812, 18979, 19146,
       19312, 19479, 19646, 19810, 19980, 20146, 20313, 20479, 20646,
       20813, 20980,

In [49]:
len(song.data)

33659324